In [2]:
import datetime
import numpy as np
#import scipy as sp
#import uuid
import hashlib
import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import os
import re
import math
from datetime import date
import ftplib
from ftplib import FTP
import gzip as gz
import collections
from collections import deque,OrderedDict,namedtuple
import sys
import cProfile
import multiprocessing
import inspect
#https://github.com/chaithemleib/intervaltree - don't use one that cannot support objects attached to intervals
import yaml
import datetime
import time
import ast
import codecs
import uuid
import json
# add liftover
from pyliftover import LiftOver
import hgvs.parser
import hgvs.dataproviders.uta
import hgvs.assemblymapper
import hgvs.projector
# internet
import urllib3
import certifi
import xmlrpc.client

In [3]:
# Notebook display options
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 1000)
pd.set_option('max_colwidth',1000)
%matplotlib inline
%autosave 0

Autosave disabled


In [4]:
hdp = hgvs.dataproviders.uta.connect()
variantmapper = hgvs.variantmapper.VariantMapper(hdp)
am = hgvs.assemblymapper.AssemblyMapper(hdp,assembly_name='GRCh37', alt_aln_method='splign',replace_reference=True)
hp = hgvs.parser.Parser()
hn = hgvs.normalizer.Normalizer(hdp)
vr = hgvs.validator.Validator(hdp=hdp)

# genomic coords tester

In [5]:
hgvs_g = 'NC_000007.13:g.36561662C>T'

# cDNA coord tester

In [6]:
hgvs_c = 'NM_001637.3:c.1582G>A'

# parse genomic

In [7]:
var_g = hp.parse_hgvs_variant(hgvs_g)

In [8]:
var_g

SequenceVariant(ac=NC_000007.13, type=g, posedit=36561662C>T, gene=None)

In [9]:
var_g.posedit.pos.start

SimplePosition(base=36561662, uncertain=False)

In [10]:
str(var_g)

'NC_000007.13:g.36561662C>T'

# get transcripts of a gene

In [11]:
transcripts = am.relevant_transcripts(var_g)

In [12]:
sorted(transcripts)

['NM_001177506.1', 'NM_001177507.1', 'NM_001637.3']

# project a genomic variant onto a cDNA variant

In [ ]:
var_c = am.g_to_c(var_g, 'NM_001637.3')

In [ ]:
var_c

In [ ]:
var_c2 = am.g_to_c(var_g, 'NM_001177507.1')
var_c2

In [ ]:
var_c3 = am.g_to_c(var_g,'NM_001177506.1')
var_c3

In [ ]:
str(var_c)

In [ ]:
str(var_c2)

In [ ]:
str(var_c3)

In [ ]:
#dir(var_c3)

# validate a variant

In [ ]:
var_c3.validate

In [ ]:
try:
    # shift the NM_001177506.1 position +2 1582 to 1584 and see if this validates
    vr.validate( hp.parse_hgvs_variant('NM_001177506.1:c.1584G>A') )
    vr.validate( hp.parse_hgvs_variant('NM_001637.3:c.1582G>A') )
except hgvs.exceptions.HGVSError as e:
    print(e)

In [ ]:
var_p = am.c_to_p(var_c)

In [ ]:
var_p

In [ ]:
 str(var_p)

In [ ]:
var_p.posedit.uncertain = False

In [ ]:
str(var_p)

In [ ]:
 var_p.format(conf={"p_3_letter": False})

# Normalize a variant 

In [ ]:
hn.normalize(hp.parse_hgvs_variant('NM_001166478.1:c.35_36insT'))

In [ ]:
c1 = hp.parse_hgvs_variant('NM_001166478.1:c.31del')

In [ ]:
c1

In [ ]:
c1n = hn.normalize(c1)

In [ ]:
c1n

In [ ]:
g = am.c_to_g(c1)

In [ ]:
g

In [ ]:
gn=am.c_to_g(c1n)

In [ ]:
gn

In [ ]:
c2 = am.g_to_c(g, c1.ac)

In [ ]:
c2

In [ ]:
var_c1 = hgvsparser.parse_hgvs_variant('NM_001261456.1:c.1762A>G')

In [ ]:
pj = hgvs.projector.Projector(hdp=hdp,
                              alt_ac='NC_000001.10',
                              src_ac=var_c1.ac,
                              dst_ac='NM_001261457.1')

In [ ]:
pj.project_variant_forward(var_c1)

In [ ]:
hdp = hgvs.dataproviders.uta.connect()
variantmapper = hgvs.variantmapper.VariantMapper(hdp)
hgvsparser = hgvs.parser.Parser()

# project cDNA onto protein

In [ ]:
var_c1 = hgvsparser.parse_hgvs_variant('NM_001261456.1:c.1762A>G')
var_p1 = variantmapper.c_to_p(var_c1, None)
var_c1, var_p1

In [ ]:
var_g = variantmapper.c_to_g(var_c1,'NC_000001.10')
var_g

# get info manually on a variants within genes

In [ ]:
txs = hdp.get_tx_for_gene('LY9')
txs

In [ ]:
txs = hdp.get_tx_for_gene('ABCA4')
txs

In [ ]:
var_c2 = variantmapper.g_to_c(var_g,'NM_001261457.1',alt_aln_method='splign')
var_p2 = variantmapper.c_to_p(var_c2, None)
var_c2, var_p2

# Normalize a structural variant

In [ ]:
v = hp.parse_hgvs_variant("NM_001166478.1:c.30_31insT")
str(hn.normalize(v))

# DATA! : test this on a real data-set

In [13]:
dt={'ALLELEID': int,'VariationID': int, 'SCV': str, 'Submitter': str, 'chrom': str, 'pos': int, 'ref': str, 'alt': str, 'CLNHGVS': object}
hgvs_df=pd.read_csv('/mnt/c/Users/mcgold/DATA/HGVStestFile.tsv',dtype=dt,sep="\t",header=0)

# put this together and check a file that has a column of HGVS g data

In [ ]:
# make a structure to hold a validation round when given a genomic HGVS
# validate the genomic, cDNA/s and protein/s
# validate the normalizations of genomic, cDNA and protein
attr=['ID','assembly','var_g','var_c','var_p','isNormalized_g','isNormalized_c',
      'isNormalized_p','isValid_g','isValid_c','isValid_p','transcripts','xrefs','timestamp']
Validation = namedtuple('variant',attr) 
n_attr=len(attr)
priorID=0
tests_dict={}
d=datetime.datetime.today().strftime('%Y-%m-%d')
date=str(d)
assembly='GRCh37'
for nth,i in enumerate(hgvs_df.index):
    ID=hgvs_df.loc[i,'ALLELEID']
    if ID != priorID:
        CLNHGVS_list=ast.literal_eval(hgvs_df.loc[i,'CLNHGVS'])
        record=Validation._make(['.']*n_attr)
        record._replace(ID=ID,timestamp=date)
        tests_dict[ID]=record
        tests_dict[ID]=tests_dict[ID]._replace(ID=ID)
        tests_dict[ID]=tests_dict[ID]._replace(assembly='GRCh37')
        tests_dict[ID]=tests_dict[ID]._replace(timestamp=date)
#    sys.stdout.write(str(ID) + "\t" + str(CLNHGVS_list) + "\n")
# genomic data
        for g in CLNHGVS_list:
#            print(nth,g)
            var_g='.'
            transcripts='.'
            varPlst=[]
            tests_dict[ID]=tests_dict[ID]._replace(var_g=g)
# validate the genomic variant 
            try:
                var_g=hp.parse_hgvs_variant(str(g))
                x=vr.validate( hp.parse_hgvs_variant(str(var_g)))
                tests_dict[ID]=tests_dict[ID]._replace(isValid_g=x)
            except:
                tests_dict[ID]=tests_dict[ID]._replace(isValid_g=None)
# check normalization
            try:
                x=hn.normalize(hp.parse_hgvs_variant(str(var_g)))
            except:
                x=None
            if str(x)==str(var_g):
                tests_dict[ID]=tests_dict[ID]._replace(isNormalized_g=True)
            elif (x is not None) and (str(x)!=str(var_g)):
                tests_dict[ID]=tests_dict[ID]._replace(isNormalized_g=False)
            else:
                tests_dict[ID]=tests_dict[ID]._replace(isNormalized_g=None)
# get the transcripts and check validity and normalization
            try:
                transcripts = sorted(am.relevant_transcripts(var_g))
                transcriptStr=";".join(transcripts)
                tests_dict[ID]=tests_dict[ID]._replace(transcripts=transcriptStr)
                sys.stdout.write(str(ID) + "\t" + str(CLNHGVS_list) + "\t" + str(transcripts) + "\n")
            except:
                tests_dict[ID]=tests_dict[ID]._replace(transcripts=None)
            try:
                tests_dict[ID]=tests_dict[ID]._replace(var_p=varPstr)
                xrefStr=transcriptStr + ';' + varPstr
                tests_dict[ID]=tests_dict[ID]._replace(xrefs=xrefStr)
            except:
                pass  
            vars_p=[]
            vars_c=[]
            norm_p=[]
            norm_c=[]
            isvalid_cs=[]
            isvalid_ps=[]
            for t in transcripts:
                if t!='.':
                    var_c=am.g_to_c(var_g, str(t))
                    var_p=variantmapper.c_to_p(var_c, None)
                    varPlst.append(str(var_p))
# for all the linked transcripts and proteins check the normalization
                    try:
                        x=hn.normalize(hp.parse_hgvs_variant(str(var_c)))
                        vars_c.append(str(x))
                    except:
                        x=None
                    try:
                        y=hn.normalize(hp.parse_hgvs_variant(str(var_p)))
                        vars_p.append(str(y))
                    except:
                        y=None                    
                    if str(x)==str(var_c):
                        sys.stdout.write("\tNORMALIZATION PASS: " +  str(x) +"=" + str(var_c) + "\n")
                        norm_c.append(str(True))
                    elif (x is not None) and (str(x)!=str(var_c)):
                        sys.stderr.write("\tNORMALIZATION FAIL: " +  str(x) +"=" + str(var_c) + "\n")
                        norm_c.append(str(False))
                    else:
                        sys.stdout.write("\tNORMALIZATION NULL: " +  str(x) +"=" + str(var_c) + "\n")
                        norm_c.append('.')
                    if str(y)==str(var_p):
                        sys.stdout.write("\tNORMALIZATION PASS: " +  str(y) +"=" + str(var_p) + "\n")
                        norm_p.append(str(True))
                    elif (y is not None) and (str(y)!=str(var_c)):
                        sys.stderr.write("\tNORMALIZATION FAIL: " +  str(y) +"=" + str(var_p) + "\n")
                        norm_p.append(str(False))
                    else:
                        sys.stdout.write("\tNORMALIZATION NULL: " +  str(y) +"=" + str(var_p) + "\n")
                        norm_p.append('.')
# for all linked transcripts  and proteins check validity
                    try:
                        x=vr.validate(hp.parse_hgvs_variant(str(var_c)))
                        vars_c.append(str(x))
                    except:
                        x=None
                    isvalid_cs.append('.')
                    try:
                        y=vr.validate(hp.parse_hgvs_variant(str(var_p)))
                        isvalid_ps.append(str(y))
                    except:
                        y=None                    
                    isvalid_ps.append('.')
            vars_cStr=";".join(vars_c)
            vars_pStr=";".join(vars_p)
            norm_cStr=";".join(norm_c)
            norm_pStr=";".join(norm_p)
            valid_cStr=";".join(isvalid_cs)
            valid_pStr=";".join(isvalid_ps)

            tests_dict[ID]=tests_dict[ID]._replace(var_c=vars_cStr)
            tests_dict[ID]=tests_dict[ID]._replace(var_p=vars_pStr)
            tests_dict[ID]=tests_dict[ID]._replace(isNormalized_c=norm_cStr)
            tests_dict[ID]=tests_dict[ID]._replace(isNormalized_p=norm_pStr)            
            tests_dict[ID]=tests_dict[ID]._replace(isValid_c=valid_cStr)
            tests_dict[ID]=tests_dict[ID]._replace(isValid_p=valid_pStr)
        priorID=ID

96938	['NC_000013.10:g.32918712_32918716del']	['NM_000059.3']
	NORMALIZATION PASS: NM_000059.3:c.6859_6863del=NM_000059.3:c.6859_6863del
	NORMALIZATION PASS: NP_000050.2:p.(Arg2287LeufsTer4)=NP_000050.2:p.(Arg2287LeufsTer4)
102767	['NC_000013.10:g.32937324C>A']	['NM_000059.3']
	NORMALIZATION PASS: NM_000059.3:c.7985C>A=NM_000059.3:c.7985C>A
	NORMALIZATION PASS: NP_000050.2:p.(Thr2662Lys)=NP_000050.2:p.(Thr2662Lys)


Normalization of intronic variants is not supported; returning unnormalized variant


231894	['NC_000013.10:g.32921037A>G']	['NM_000059.3']
	NORMALIZATION NULL: None=NM_000059.3:c.7007+4A>G
	NORMALIZATION PASS: NP_000050.2:p.?=NP_000050.2:p.?


Normalization of intronic variants is not supported; returning unnormalized variant


66722	['NC_000013.10:g.32900753A>G']	['NM_000059.3']
	NORMALIZATION NULL: None=NM_000059.3:c.631+3A>G
	NORMALIZATION PASS: NP_000050.2:p.?=NP_000050.2:p.?
180681	['NC_000013.10:g.32953990A>G']	['NM_000059.3']
	NORMALIZATION PASS: NM_000059.3:c.9057A>G=NM_000059.3:c.9057A>G
	NORMALIZATION PASS: NP_000050.2:p.(Lys3019=)=NP_000050.2:p.(Lys3019=)


...

46477	['NC_000013.10:g.32900291del']	['NM_000059.3']
	NORMALIZATION NULL: None=NM_000059.3:c.475+4del
	NORMALIZATION PASS: NP_000050.2:p.?=NP_000050.2:p.?


Normalization of intronic variants is not supported; returning unnormalized variant


432828	['NC_000013.10:g.32954284T>C']	['NM_000059.3']
	NORMALIZATION NULL: None=NM_000059.3:c.9256+2T>C
	NORMALIZATION PASS: NP_000050.2:p.?=NP_000050.2:p.?


Normalization of intronic variants is not supported; returning unnormalized variant


476532	['NC_000013.10:g.32893213G>A']	['NM_000059.3']
	NORMALIZATION NULL: None=NM_000059.3:c.68-1G>A
	NORMALIZATION PASS: NP_000050.2:p.?=NP_000050.2:p.?


In [17]:
tests_dict

{96938: variant(ID=96938, assembly='GRCh37', var_g='NC_000013.10:g.32918712_32918716del', var_c='NM_000059.3:c.6859_6863del;True', var_p='NP_000050.2:p.(Arg2287LeufsTer4)', isNormalized_g=True, isNormalized_c='.', isNormalized_p='.', isValid_g=True, isValid_c='.', isValid_p='True;.', transcripts='NM_000059.3', xrefs='.', timestamp='2020-10-07'),
 102767: variant(ID=102767, assembly='GRCh37', var_g='NC_000013.10:g.32937324C>A', var_c='NM_000059.3:c.7985C>A;True', var_p='NP_000050.2:p.(Thr2662Lys)', isNormalized_g=True, isNormalized_c='.', isNormalized_p='.', isValid_g=True, isValid_c='.', isValid_p='True;.', transcripts='NM_000059.3', xrefs='.', timestamp='2020-10-07'),
 231894: variant(ID=231894, assembly='GRCh37', var_g='NC_000013.10:g.32921037A>G', var_c='', var_p='NP_000050.2:p.?', isNormalized_g=True, isNormalized_c='.', isNormalized_p='.', isValid_g=True, isValid_c='.', isValid_p='True;.', transcripts='NM_000059.3', xrefs='.', timestamp='2020-10-07'),
 66722: variant(ID=66722, ass

In [18]:
pd.DataFrame.from_dict(tests_dict,orient='index').to_csv('/mnt/c/Users/mcgold/DATA/HGVSValidationResult_part_20201007.tsv',sep="\t",header=True,index=False)

In [ ]:
df=pd.DataFrame.from_dict(tests_dict,orient='index')

In [ ]:
df